# <a id='toc1_'></a>[**MICRO-452 Final Report**](#toc0_)
### <a id='toc1_1_1_'></a>[*Group 45: Anton Balykov, Teo Halevi, Cyprien Lacassagne, Seonmin Park*](#toc0_)

# <a id='toc2_'></a>[Table of Contents](#toc0_)


- [Project Objective](#toc3_)    
- [I. Computer Vision](#toc4_)    
  - [I.1 Introduction](#toc4_1_)    
  - [I.2 Camera Set Up](#toc4_2_)    
  - [I.3 Global Obstables and Thymio Detection](#toc4_3_)    
- [II. Global Navigation](#toc5_)    
  - [II.1 Introduction](#toc5_1_)    
  - [II.2 Path Planning](#toc5_2_)    
  - [II.3 Algorithm](#toc5_3_)    
- [III. Local Navigation](#toc6_)    
  - [III.1 Introduction](#toc6_1_)    
  - [III.1 Local Avoidance](#toc6_2_)    
- [IV. Filtering](#toc7_)    
  - [IV.1 Introduction](#toc7_1_)    
  - [IV.2 Methodology](#toc7_2_)    
  - [IV.3 Dynamics of Thymio](#toc7_3_)    
  - [IV.4 Design and Implementation of Filter](#toc7_4_)    
  - [IV.5 Simulation](#toc7_5_)    
- [Conclusion](#toc8_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc3_'></a>[Project Objective](#toc0_)

This project leverages the Thymio robot to explore and integrate key concepts introduced in the Introduction to Mobile Robotics course. We hereby focus on implementing and demonstrating core components: computer vision, global navigation, motion control, local navigation, and filtering. The mission of the robot is to navigate itself from the designated start position to the specified goal position, successfully avoiding fixed obstacles as well as dynamically placed one. Achieving this requires precise motion control, robust obstable detection, and efficient filtering technique. Such practical application of the five above-mentioned elements would reflect our team's comprehensive understanding of mobile robotics principles.

# <a id='toc4_'></a>[I. Computer Vision](#toc0_)

## <a id='toc4_1_'></a>[I.1 Introduction](#toc0_)

bla bla bla

## <a id='toc4_2_'></a>[I.2 Camera Set Up](#toc0_)

bla bla bla

## <a id='toc4_3_'></a>[I.3 Global Obstables and Thymio Detection](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc5_'></a>[II. Global Navigation](#toc0_)

## <a id='toc5_1_'></a>[II.1 Introduction](#toc0_)

One main component of this project is navigation. Here, we discuss about our implementation of the global navigation, whose main objective is to find a collision-free path from a source to a goal, on a map filled with polygons acting as obtacles. The environment of motion is imply a 2D flat surface such as the ground or a table. The global, and thus **persistant** obstacles are red polygons made of paper, and placed freely by hand to test the robustness of the path planning algorithm. The boundaries of our map are determined by the field of view - and thus the height - of the camera from the ground, as regions that lie outside the visible field are not processed. To find the optimal path from the robot to the goal, the strategy relies on a metric map and visibility graph, as well as Dijkstra's graph search algorithm.

Global navigation is only involved in the beginning of the program, outside of the main loop. The only case where global path is updated is when the robot is "kidnapped", i.e. moved by hand to another position on the map.

## <a id='toc5_2_'></a>[II.2 Initial Strategy](#toc0_)

Initially, we decided to go with a discretized fixed-size cell-map and use Dijktra's search algorithm. Although the implementation was not difficult, we thought that it was not as elegant as the more intuitive visibility graph approach. Nonetheless, here is the result of this preliminary work:

![Cell_decomposition](report-images/cell_decomposition.png)

## <a id='toc5_3_'></a>[II.3 Polygon Expansion](#toc0_)

We decided to clearly separate features detection (`ComputerVision`) and path planning (`Navigation`).
The class `Navigation` is constructed by receiving the position of the robot (source), of the goal, and of the vertices of each obstacle. All of this data is extracted from the camera image by the class `ComputerVision`, as explained in Part I. Each obstacle conists of a list of points in the form `[x, y]`.

In [ ]:
import numpy as np
import pyvisgraph as vg

THYMIO_RADIUS = 70 # mm

class Navigation:
    def __init__(self, obstacles, robot, goal):
        """
        :param obstacles: A list of all the obstacles on the map. An obstacle is a list of 
                            vertices, ordered CCW.
        :param robot: The position of the robot.
        :param goal: The position of the goal.
        """
        self.obstacles = obstacles
        self.obstacles_count = len(obstacles)
        self.extended_obstacles = []
        self.source = robot
        self.goal = goal
        self.path = []

Before delving into graph creation, the polygons need to be expanded in order to account for the size of the robot. Otherwise, each obstacle's vertex would be registered as a node, and thus a potential waypoint in the final path, and would be mapped to the center of the robot. This is not ideal since the robot would bump into the obstacles. The function `augment_obstacles` loops through the polygons and computes two new points for each vertex, expanded once in the direction of each of the edges that make up the vertex, and once in the direction of their normals, by a constant factor `THYMIO_RADIUS`, which is the radius of the robot in mm. That way, we ensure the robot always keeps a safe minimal distance with the obstacles, even when some portion of the resulting path is tengential to a polygon edge.

In [ ]:
@staticmethod
def augment_obtacles(self):
    """
    Augments the countour of the obstacles w.r.t. the radius of the robot.
    """
    for polygon in self.obstacles:
        extended_polygon = []

        count = len(polygon)
        for i in range(count):
            vertex = polygon[i]
            prev = polygon[i-1]
            next = (polygon[i+1] if i < count-1 else polygon[0])

            edge1 = np.subtract(vertex, prev)
            edge2 = np.subtract(next,vertex)

            dir1 = np.array(edge1 / np.linalg.norm(edge1))
            dir2 = np.array(edge2 / np.linalg.norm(edge2))

            perp1 = -np.array([dir1[1], -dir1[0]])
            perp2 = -np.array([dir2[1], -dir2[0]])

            v1 = vertex + THYMIO_RADIUS * dir1 + THYMIO_RADIUS * perp1
            v2 = vertex - THYMIO_RADIUS * dir2 + THYMIO_RADIUS * perp2

            # Preserve the winding of the original polygon
            angle = np.arccos(np.clip(np.dot(-dir1, dir2), -1.0, 1.0))
            if angle <= np.pi / 2.0:
                extended_polygon.append(v1)
                extended_polygon.append(v2)
            else:
                extended_polygon.append(v2)
                extended_polygon.append(v1)

        self.extended_obstacles.append(extended_polygon)

Here is the result of our implemantation of polygon expansion. In this example, we set the vertices and thymio's radius to arbitrary values.

![Polygon_expansion](report-images/Polygon_expansion.png)

Expanding the vertices by two points instead of only one, allows for a smoother bypassing and reduces the total path length, while increasing the accuracy of the Kalman filter.

## <a id='toc5_4_'></a>[II.4 Visibility Graph & Graph Search](#toc0_)

Then, a visibility graph is built using the open-source library [pyvisgraph](https://github.com/TaipanRex/pyvisgraph). This library uses [Lee's algorithm](https://dav.ee/papers/Visibility_Graph_Algorithm.pdf), O(n²log(n)), to compute the visibility of each node.<br>
This operation will fail if the winding of the obstacles is incoherent. That is why we take care of expanding the corners of the obstacles in the right order.

Finally, the shortest path is computed using Dijkstra's algorithm, and returned to the main function.

In [ ]:
def get_shortest_path(self):
    """
    Computes the shortest path from source (robot) to goal, while avoiding the obstacles.

    :returns:
        A list of the coordinates of each node from robot position to goal.
    """
    self.augment_obtacles(self)

    graph = vg.VisGraph()
    polygons = []
    for obstacle in self.extended_obstacles:
        polygon = []
        for point in obstacle:
            polygon.append(vg.Point(point[0], point[1]))

        polygons.append(polygon)
            
    graph.build(polygons)
    path = graph.shortest_path(vg.Point(self.source[0], self.source[1]), vg.Point(self.goal[0], self.goal[1]))

    for point in path:
        self.path.append(point)
        print(point)

    return self.path

 ## <a id='toc5_5_'></a>[II.5 Results](#toc0_)

Below are the results of two different configurations that shows how the path changes when the first shortest path (left) is blocked (right). A pass through between two obstacles is choked when at least one vertex of a polygon lies into the expanded version of the other one, as you can see on the right image, where we lowered the middle triangle by 50 units.

![path_planning_1](report-images/path_planning_1.png) ![path_planning_2](report-images/path_planning_2.png)
 

# <a id='toc6_'></a>[III. Local Navigation](#toc0_)

## <a id='toc6_1_'></a>[III.1 Introduction](#toc0_)

bla bla bla

## <a id='toc6_2_'></a>[III.1 Local Avoidance](#toc0_)

bla bla bla

.

.

.

.

# <a id='toc7_'></a>[IV. Filtering](#toc0_)

## <a id='toc7_1_'></a>[IV.1 Introduction](#toc0_)


There is a need for a method that compensates the defect that there is no direct mapping between Thymio's sensors and the values which are used for control. The absolute position of the robot can be obtained directly from the camera and global path planning techniques, but the robot may have low precision and update rate, causing drifts when the velocity or acceleration is integrated. We can address this challenge by implementing Kalman filter as it provides a sound framework to use all available measurements in an optimal way. It relies on a model for the system structure and the effect of disturbances both on the state and on the measurements.

The Kalman filter we implemented estimates the position and speed of the robot when it moves along the planned path. Specifically, we implemented extended Kalman filter to take into account the system involves nonlinear dynamics that would need to be linearized around the current estimate, focusing on local optimality.


## <a id='toc7_2_'></a>[IV.2 Methodology](#toc0_)

The Thymio robot moves at constant speed across a pattern made of black and white stripes of 50 mm, identical to one of the activities found in Exercise 8 of the course. The actual mean velocity of the two wheels and the mean reflected light read by the two ground sensors are sent in an event at a constant rate. The python code receives these values, updates the estimated velocity and position of the robot, and displays them continuously in a plot.


To obtain velocity parameters, we began with estimating Thymio's speed by let it cross seven blocks that are 50 mm each with a sample rate of 0.1 seconds and marked peaks of the ground sensor meaasuremnet indicating that it has crossed evey block. With known and obtained information, we could obtain the conversion factor of 0.4 to conver Thymio's speed to mm/s.

![Find Peaks](find_peaks.png "Fine Peaks")

Now, to obtain speed variance, we observed the data where the robot was moving by plotting the robot's speed and computed average velocity and thus the speed variance by diving the average speed with the robot's speed in mm/s.

![Measured Vel](measured_vel.png "Measured Vel")

With the assumption that half of the variance is cause by the measurement and the other half is caused by the perturbation to the states, we obtain 

$q_v = r_v = 16.08621$ $mm^2/s^2$ $\space$ where $\space$ $\sigma^2_{vel} = 32.17242$ $mm^2/s^2$

Variances on position state and measurement as well as those for the angle for orientation ($\sigma^2_{pos}$ and $\sigma^2_{\theta}$) were an arbitrary value that were tuned manually for the specific Thymio robot that is going to be used for demonstration.

## <a id='toc7_3_'></a>[IV.3 Dynamics of Thymio](#toc0_)

To begin with the implementation, the state space model is

$\hat{x}_{k-1|k-1} =  \begin{bmatrix}
                        x_{pos_{k-1}} \\
                        y_{pos_{k-1}} \\
                        \theta_{k-1} \\
                        v_{left_{k-1}} \\
                        v_{right_{k-1}} \\
                        \end{bmatrix}$

Under control input vector,

$ u =[v_{trans}, v_{rot}] $, $\space$ where $\space$ $v_{tran} = \frac{v_{left} + v_{right}}{2} $ $\space$ and $\space$ $ v_{rot} = \frac{v_{left} - v_{right}}{l_{base}} $

Then, we can see that the robot's new position and orientation are predicted in the following way

$x_{pos_{k}} = x_{pos_{k-1}} + v_{trans_{k}} \cdot \cos(\theta_{k-1}) \cdot dk$

$y_{pos_{k}} = y_{pos_{k-1}} + v_{trans_{k}} \cdot \sin(\theta_{k-1}) \cdot dk$

$\theta_{k} = \theta_{k-1} + v_{rot_{k}} \cdot dk$

## <a id='toc7_4_'></a>[IV.4 Design and Implementation of Filter](#toc0_)



We have introduced that the extended Kalman filter takes into account nonlinear system dynamics in the previous sections. In order to make prediction of the next state, We use a nonlinear motion model $f(x, u)$. That is, we can formulate their relation as:

$\hat{x}_{k|k-1} = f(\hat{x}_{k-1|k-1}, u_k)$

Our $f(x, u)$ is then introduced as

$f(\hat{x}_{k-1|k-1}, u_k) = \begin{bmatrix}
1 & 0 & 0 & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & 0 & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{\text{base}}} & \frac{dk}{2 \cdot l_{\text{base}}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

In this prediction step of the filter, we make a use of the predicted covariance matrix $P_{k-1|k-1}$.

$P_{k-1|k-1} = \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

as each column corresponds to the state variables, from the first column to the last column, being $x_{pos_{k-1}}$, $y_{pos_{k_1}}$, $\theta_{k-1} $, $v_{left_{k-1}}$, and $v_{right_{k-1}}$, respectively.

The updated (current) covariance matrix is as follows:

$P_{k|k-1} = F_{k}P_{k-1|k-1}F_{k}^{T} + Q_{k}$

where $F_{k}$ is the Jacobian of $f_{k}$ matrix, and $Q_{k}$ is the noise covariance matrix of the state model. The $Q_{k}$ matrix specifically represents the degree or the extend of how much the actual motion of Thymio deviates from its assumed state space model which would be the path that it should follow for our case.

We can write the two matrices as below:

$F_{k|k-1} = 
\begin{bmatrix}
1 & 0 & v_{trans} \cdot \sin(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk & -0.5 \cdot \cos(\theta_{k-1}) \cdot dk \\ 
0 & 1 & v_{trans} \cdot \cos(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk & 0.5 \cdot \sin(\theta_{k-1}) \cdot dk \\ 
0 & 0 & 1 & -\frac{dk}{2 \cdot l_{base}} & \frac{dk}{2 \cdot l_{base}} \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

$Q_{k|k-1} = \alpha \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

where $\alpha$ is a scaling factor, which we have initialized as $\alpha =10$, such that 

$Q_{k-1|k-1} = 10 \cdot 
\begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

We set $\alpha$ large enough so that the filter tracks large changes in the wheel sensor measurements more closely as if it was smaller.

In the updating process of the extended Kalman filter, we make a use of the term *innovation* which is the difference between the actual measurements and predicted observations. Let $z_{k}$ be the observation vector and $i_{k}$ be the innovation term. Then, we can compute:

$i_{k} = z_{k} - H_{k}\hat{x}_{k-1}$ 

where $H_{k}$ is the measurement matrix that is used to convert the predicted state estimate at time $k$ into predicted sensor measurements at time $k$.

In this project, we have two cases that can happen. One of them is when Thymio gets information from the camera, we name it as "when it has vision," and the other is when it does not get information from the camera, "when it has no vision," specifically when the camera is covered.

When it has vision, 

$H_{k} = \begin{bmatrix}
1 & 0 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 & 0 \\ 
0 & 0 & 1 & 0 & 0 \\ 
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

When it does not have vision, 

$H_{k} = \begin{bmatrix}
0 & 0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 0 & 1 
\end{bmatrix}$

as the robot will have to rely only on its wheel sensor measurements in this scenario.

With the innovation $y_{k}$, we obtain the innovation covariance matrix $S_{k}$ to be used for the computation of Kalman Gain $K_{k}$.

$S_{k} = H_{k}P_{k|k-1}H_{k}^{T} + R_{k}$

where $R_{k}$ is the sensor measurement noise covariance matrix. It also has different dimensionality depending on if Thymio has vision or not.

When it has vision,

$R_{k} = \beta \cdot \begin{bmatrix}
\sigma^2_{pos} & 0 & 0 & 0 & 0 \\ 
0 & \sigma^2_{pos} & 0 & 0 & 0 \\ 
0 & 0 & \sigma^2_{\theta} & 0 & 0 \\ 
0 & 0 & 0 & \sigma^2_{vel} & 0 \\ 
0 & 0 & 0 & 0 & \sigma^2_{vel} 
\end{bmatrix}$

When it does not have vision,

$R_{k} = \gamma \cdot \begin{bmatrix}
\sigma^2_{vel} & 0\\ 
0 & \sigma^2_{vel} 
\end{bmatrix}$

with the same logic of $H_{k}$ matrices, and constants $\beta = 1$ and $\gamma = 1$ were defined through trial and error experiments just like $\alpha$ of the $Q_{k}$ matrix.

The Kalman Gain matrix is computed as follows:

$K_{k} = P_{k|k-1}H_{k}^{T}S_{k}^{-1}$

and we can update our estimated state and its covariance matrix by

$\hat{x}_{k|k} = \hat{x}_{k|k-1} + K_{k}i_{k}$ $\space$ and $\space$ $ P_{k|k} = (I - K_{k}H_{k})P_{k|k-1}$ $\space$,  respectively.


The above steps are implemented as a class ***KalmanFilterExtended*** and the class is called in the **main.py** file where all the five components of the project are integrated.

The function ***run_ekf()*** ensures that the extened filter is intiated and the snippet from **main.py** below ensures that the filter is running in a loop throughout the entire mission of the robot.

Additionally, we have set parameters that defines *kidnapping* in terms of the difference in the Thymio's orientation and position exceeding the threshold values. Specifically, we have set the threshold for orientation and position to be 35cm and 60°, and exceeding either (or both) of these values would be considered getting kidnaped.

**Codes to be displayed here **

## <a id='toc7_5_'></a>[IV.5 Simulation](#toc0_)

to be shown later

# <a id='toc8_'></a>[Conclusion](#toc0_)
It has been an incredible journey to embark on this project within the constrained timeline of just three weeks, replete with challenges and invaluable learning opportunities. From the very beginning, we have had to face numerous obstacles, such as technical errors and conceptual hurdles, which have pushed us to refine our problem-solving skills and deepen our understanding of mobile robotics. Despite these challenges, our team has been able to overcome setbacks and achieve results we are proud of through persistence, discipline, and a collaborative spirit.

Working through the complexities of the project was as rewarding as it was demanding. From understanding the theoretical basis to the implementation of practical solutions, each step provided an opportunity to reinforce and showcase our grasp of the principles of mobile robotics. The hands-on nature of the project tested not only our knowledge but also helped us grow as engineers, fostering resilience, adaptability, and creativity in tackling real-world problems.

We would like to express our deep gratitude to everyone who guided and supported us through this journey. We would like to thank Professor Mondada, who inspired us with his experience and vision, and the teaching assistants, who were always ready to help, explain doubts, and give us valuable insights. Their encouragement and mentorship played a very important role in driving the success of our project.

It is not only an academic milestone but also a source of personal and professional growth, and we will be carrying the lessons learned herein as we progress in our future endeavors related to robotics and beyond.